In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
df_train=train.drop(['ID_code', 'target'],axis=1)
df_test=test.drop('ID_code',axis=1)
Id_train=train.ID_code
Id_test=test.ID_code
                 

In [ ]:
import gc
target=train.target
del train,test
gc.collect()

# Assumption for logistic regresson
## 1/4 First, binary logistic regression requires the dependent variable to be binary(obviously)


## 2/3  Second, logistic regression requires the observations to be independent of each other.  In other words, the observations should not come from repeated measurements or matched data. In this case, observations are some kind of measures for different customers,  **** so we may accept this one.

## 3/4 Check the correlations of features.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = df_train.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


In [ ]:
f, axes = plt.subplots(2, 2, figsize=(10,10), sharex=False)
sns.despine(left=True)
sns.heatmap(corr.iloc[0:49,0:49], center=0,
            #cmap=cmap,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=axes[0, 0])
sns.heatmap(corr.iloc[50:99,50:99], center=0,
            #cmap=cmap,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=axes[0, 1])
sns.heatmap(corr.iloc[100:149,100:149], center=0,
            #cmap=cmap,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=axes[1, 0])
sns.heatmap(corr.iloc[150:199,150:199], center=0,
            #cmap=cmap,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, ax=axes[1, 1])

Seems there is no highly correlated features.
(1. What if there are some high colinearty?  PCA-> Orthognal features).

# 4/4  f(Independent variables)  are linearly related to the log odds.
## In most case, f(X)=X*beta, where beta is coffecient of the features (like linear regresson).  I use MLE estimator  to test this assumption. (In another words, solve model first and check the correlation of X*beta and log odd(log(p/(1-p), p is the probability of Y to be 1)

In [ ]:
#feature scaling
df_train -= df_train.min() 
df_train /= df_train.max() 
df_test -= df_test.min()  # equivalent to df = df - df.min()
df_test /= df_test.max() 

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='sag',fit_intercept=True,C=1000,
                          multi_class='ovr').fit(df_train, target)
target_hat=clf.predict(df_train)

In [ ]:
np.corrcoef(target_hat,target)

Correlation coefficient is 0.398, not that bad. This implay that there are some linear relationshape between target and (X*theta).

# Use L1, L2 control complexity (bais and variance tradeoff ) of Log-regression model. Use 5 folds cross validation choose the proper hyperparameter.

from sklearn.model_selection import StratifiedKFold
<br>
from sklearn.model_selection import GridSearchCV
<br>
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
<br>
parameters1 = {'C':np.logspace(-9, 0, num=50, endpoint=False, base=10.0).tolist()} 
<br>
lgr = LogisticRegression(random_state=0, solver='sag',fit_intercept=True, multi_class='ovr')
<br>
GRD1= GridSearchCV(lgr, parameters1,scoring='roc_auc',cv=5)
<br>
GRD1.fit(df_train, target)
<br>
print(GRD1.best_score_,GRD1.best_params_)
<br>
0.8595038749527413 {'C': 0.05495408738576237}
<br>

#### Best C is  0.05495408738576237. Note that  The default pernalized type is L1.

In [ ]:
clf = LogisticRegression(random_state=0, solver='sag',fit_intercept=True,C=0.05495408738576237,
                          multi_class='ovr').fit(df_train, target)
pred_class=clf.predict(df_test)
pred_prob=clf.predict_proba(df_test)[:,1]

In [ ]:
#help(GRD1.predict)
sub_class=pd.DataFrame(Id_test)
sub_prob=pd.DataFrame(Id_test)
sub_class['target']=pred_class
sub_prob['target']=pred_prob

sub_class.to_csv('Lgreg_class.csv',index=False)
sub_prob.to_csv('Lgreg_prob.csv',index=False)